# Lectures

A set of tables for handling lectures


In [289]:
#| default_exp Lectures

In [290]:
#| export
#| hide
from biobib.table import Table, sh
from biobib.sheet import Sheet
from nbdev import show_doc
import pandas as pd

In [291]:
#| hide
from fastcore.test import *
from fastcore.test import *
from fastcore.basics import *
from fastcore.foundation import *

### Define a template for lecture activities

Here is a template we can use for lecture activities. It generates a longtable which can span multiple pages, and provides the year, role, topic, venue, and place of each lecture.

In [292]:
lectures_biobib_template = r'''% UC Bio-bib Lectures Table
% Created on \VAR{created}

\begin{longtable}{lp{7.0cm}p{6.5cm}}
Month/Year & Title & Meeting/Venue (Place)\\
\hline 
\endfirsthead


\multicolumn{3}{c}%
{{\VAR{table_name} - continued from previous page }} \\ \\
Month/Year & Title & Meeting/Venue (Place)\\
\hline 
\endhead

\\
\multicolumn{3}{c}%
{{ \VAR{table_name} continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot
\BLOCK{for lecture in items}
\VAR{lecture.Month}/\VAR{lecture.Year} & \VAR{lecture.Role|tex_escape}, \VAR{lecture.Topic|tex_escape} & \VAR{lecture.Venue|tex_escape}. (\VAR{lecture.Place|tex_escape}) \\
\BLOCK{endfor}
\end{longtable}
'''

In [293]:
class Lectures(Table):
    """ 
    A Service class of Table for use in generating biobibs and CVs 
    """
    def __init__(
            self,
            sheet:Sheet=None, # Google sheet object
            worksheet:str='Lectures',   # worksheet name in google sheet
            csv_file:str=None, # optional csv file (if using csv files) 
            table_name:str=None, # Will be determined from category.
            cumulative:bool=False, # Is this table cumulative? 
            template=lectures_biobib_template # template content
    ):
        super().__init__(
            sheet=sheet,
            worksheet=worksheet, 
            table_name=table_name,
            csv_file=csv_file, 
            template=template)
        self.cumulative = cumulative
        self.df = self.clean_df()

### Provide a data cleaning function for the service table



In [294]:
#| export

@patch
def clean_df(self:Lectures,
             sort_by:list=['Year', 'Month'], # variable to sort by
             ascending:bool=[True, True] # ascending?
            )->pd.DataFrame:  # cleaned dataframe
    """
    Clean the Lectures table.
    
    """
    df = Table.table_clean_df(self)
    df = df.sort_values(by=sort_by, ascending=ascending)
    return df

In [295]:
sh.get_dataframe

<bound method Sheet.get_dataframe of <biobib.sheet.Sheet object at 0x178d88dc0>>

In [296]:
lecture_table = Lectures(sheet=sh,worksheet='Lectures',table_name='Lectures',template=lectures_biobib_template)

In [297]:
test_eq(lecture_table.table_name,'Lectures')
test_eq(list(lecture_table.df.columns),
        ['Year', 'Month', 'Role', 'Topic', 
         'Venue', 'Place', 'Eval'])

In [298]:
print(lecture_table.render_template())

% UC Bio-bib Lectures Table
% Created on 2023-06-29 19:09

\begin{longtable}{lp{7.0cm}p{6.5cm}}
Month/Year & Title & Meeting/Venue (Place)\\
\hline 
\endfirsthead


\multicolumn{3}{c}%
{{Lectures - continued from previous page }} \\ \\
Month/Year & Title & Meeting/Venue (Place)\\
\hline 
\endhead

\\
\multicolumn{3}{c}%
{{ Lectures continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot
8/2018 & Invited Speaker, Session on ``Vegetation dynamics and ecosystem resilience under global climate change'' &  Ecological Society of America Annual Meeting. (New Orleans, LA) \\
8/2018 & Instructor, Coupled Natural-Human Systems Short Course & Ridge to Reef NRT Graduate Training Program. (University of California, Irvine) \\
12/2019 & Invited Speaker, Linkages between water, vegetation, and livelihoods in Sub-Saharan drylands & American Geophysical Union. (San Francisco, CA) \\
5/2019 & Keynote Speaker, The Jack and Laura Dangermond Preserve: A Perfect Place for Advancing Conservation Sc

In [299]:
lectures_cv_template = r'''% CV Lectures Table
% Created on \VAR{created}

\begin{longtable}{l p{3.0cm} p{11.5cm}}
\endfirsthead


\multicolumn{3}{c}%
{{\VAR{table_name} - continued from previous page }} \\ \\

\endhead

\\
\multicolumn{3}{c}%
{{ \VAR{table_name} continued on next page }} \\
\endfoot

\endlastfoot
\BLOCK{for lecture in items|reversed}
\VAR{lecture.Year} & \VAR{lecture.Role|tex_escape} & \VAR{lecture.Topic|tex_escape}\BLOCK{if lecture.Venue}, \VAR{lecture.Venue|tex_escape}\BLOCK{endif}\BLOCK{if lecture.Place}, \VAR{lecture.Place|tex_escape}\BLOCK{endif} \\
\BLOCK{endfor}
\end{longtable}
'''

In [300]:
cv_table = Lectures(sheet=sh,worksheet='Lectures',table_name='Lectures',cumulative=True,template=lectures_cv_template)

In [301]:
print(cv_table.render_template())

% CV Lectures Table
% Created on 2023-06-29 19:09

\begin{longtable}{l p{3.0cm} p{11.5cm}}
\endfirsthead


\multicolumn{3}{c}%
{{Lectures - continued from previous page }} \\ \\

\endhead

\\
\multicolumn{3}{c}%
{{ Lectures continued on next page }} \\
\endfoot

\endlastfoot
2023 & Invited Speaker & The interacting impacts of shifting rainfall climatology and crop choice in smallholder production systems, Biocomplexity Institute, University of Florida \\
2023 & Keynote Speaker & Identifying and Mitigating Risk During Environmental Science Field Campaigns, Campus Safety, Health, and Environmental Management Association Annual Symposium, Santa Barbara, CA \\
2022 & Invited Lecturer & Staff Training Program: Human-caused climate change and the ecology of Santa Barbara, Santa Barbara Museum of Natural History, Santa Barbara, CA \\
2021 & Keynote Speaker & A mile wide and a pixel deep: Integrating machine learning, computer vision, and satellite imagery for coupled-natural human system mode

In [302]:
cv_table.write_template(path='../tex/CV/')

In [303]:
#| hide
import nbdev; nbdev.nbdev_export()